In [1]:
import pandas as pd
from lxml import etree

First, we want to illustrate how to use pandas' `read_xml` and XSLT, to read hierarchical data into a data frame.

In [2]:
df = pd.read_xml(
  'data.xml', 
  stylesheet='transform.xslt'
  )

The first argument is the XML data, the second argument is the XSLT stylesheet that contains the transformation.

Now, let's see how this looks:

In [3]:
df.head()

,CustomerNumber,Address
0,1,London
1,1,Hull
2,2,Birmingham
3,2,Manchester


But what happened here?

We wanted to extract the content of the DetailSubRecord, but need to also grab the customer ID so that we know which customer the data belongs to.

The XSLT contains two templates: One that simply matches the root and creates a root element, and then populates it with the output from the second template. This second template matches each DetailSubRecord element, but is able to also fetch the customer id from the parent element.

The first template, that siply creates a root:

```xml
  <xsl:template match="/root">
    <root>
      <xsl:apply-templates select="DetailRecord/DetailSubRecord"/>
    </root>
  </xsl:template>
```

The second part template:

```xml
  <xsl:template match="DetailSubRecord">
    <DetailSubRecord>
      <CustomerNumber>
        <xsl:value-of select="../CustomerNumber"/>
      </CustomerNumber>
      <Address>
        <xsl:value-of select="Address"/>
      </Address>
    </DetailSubRecord>
  </xsl:template>
</xsl:stylesheet>
```

This constructs a new xml with the structure seen above.

Let's demonstrate this by parsing the file with `lxml` directly.

In [4]:

with open('data.xml') as f:
    xml = f.read()

with open('transform.xslt') as f:
    xslt = f.read()

dom = etree.XML(xml)
xslt = etree.XML(xslt)
transform = etree.XSLT(xslt)
newdom = transform(dom)

print(str(newdom))

<?xml version="1.0"?>
<root>
  <DetailSubRecord>
    <CustomerNumber>1</CustomerNumber>
    <Address>London</Address>
  </DetailSubRecord>
  <DetailSubRecord>
    <CustomerNumber>1</CustomerNumber>
    <Address>Hull</Address>
  </DetailSubRecord>
  <DetailSubRecord>
    <CustomerNumber>2</CustomerNumber>
    <Address>Birmingham</Address>
  </DetailSubRecord>
  <DetailSubRecord>
    <CustomerNumber>2</CustomerNumber>
    <Address>Manchester</Address>
  </DetailSubRecord>
</root>



This, essentially, is what pandas reads and is able to parse. Just one, flat XML with rows directly below the root element.